In [1]:
!zenodo_get 6513631 -w gwtc2p1_files.txt
!zenodo_get 8177023 -w gwtc3p0_files.txt
!sleep 2

In [2]:
!find . -name "*gwtc*p*_files.txt" -exec grep "_nocosmo.h5" {} \; | xargs -I{} wget -P .. -nc {}

File ‘../IGWN-GWTC2p1-v2-GW190707_093326_PEDataRelease_mixed_nocosmo.h5’ already there; not retrieving.

File ‘../IGWN-GWTC2p1-v2-GW190828_063405_PEDataRelease_mixed_nocosmo.h5’ already there; not retrieving.

File ‘../IGWN-GWTC2p1-v2-GW190708_232457_PEDataRelease_mixed_nocosmo.h5’ already there; not retrieving.

File ‘../IGWN-GWTC2p1-v2-GW170809_082821_PEDataRelease_mixed_nocosmo.h5’ already there; not retrieving.

File ‘../IGWN-GWTC2p1-v2-GW190814_211039_PEDataRelease_mixed_nocosmo.h5’ already there; not retrieving.

File ‘../IGWN-GWTC2p1-v2-GW190805_211137_PEDataRelease_mixed_nocosmo.h5’ already there; not retrieving.

File ‘../IGWN-GWTC2p1-v2-GW190803_022701_PEDataRelease_mixed_nocosmo.h5’ already there; not retrieving.

File ‘../IGWN-GWTC2p1-v2-GW190719_215514_PEDataRelease_mixed_nocosmo.h5’ already there; not retrieving.

File ‘../IGWN-GWTC2p1-v2-GW190910_112807_PEDataRelease_mixed_nocosmo.h5’ already there; not retrieving.

File ‘../IGWN-GWTC2p1-v2-GW190512_180714_PEDataRelease_

In [3]:
import jax
from tqdm import tqdm
import h5py
import os
import re
import subprocess
import pandas as pd
import configparser

folder_path = "../"
write_path = "../gwtc3_converted/"

PLANCK_2018_Ho = 2.1816926176539463e-18 ### CGS
PLANCK_2018_OmegaMatter = 0.3158
PLANCK_2018_OmegaLambda = 1. - PLANCK_2018_OmegaMatter
PLANCK_2018_OmegaRadiation = 0.

In [4]:
config = configparser.ConfigParser()
config.read('../../../sampler/config.ini')
max_far = float(config["INJECTIONS"]["max_far"])

In [5]:
# Read the data
files = (file for file in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, file)))
events_files_before_far_cut = list(filter(lambda x: x.endswith((".h5", ".hdf5")), files))

to_snapshot = "snapshot-2023-11-04/GWTC/"
events_files = []

for i in tqdm(range(len(events_files_before_far_cut))):

    try:
        match = re.search(r'GW\d{6}_\d{6}', events_files_before_far_cut[i]).group(0)
        matching_files = [f for f in os.listdir(to_snapshot) if match in f][0]
        matching_json = [f for f in list(pd.read_json(to_snapshot + "GWTC.json").index) if match in f][0]
    except:
        match = re.search(r'GW\d{6}', events_files_before_far_cut[i]).group(0)

        if match in ["GW200105"]: # Get rid of cases where you've manually checked the far
            continue

        matching_files = [f for f in os.listdir(to_snapshot) if match in f][0]
        matching_json = [f for f in list(pd.read_json(to_snapshot + "GWTC.json").index) if match in f][0]


    event_df = pd.read_json(to_snapshot+matching_files)
    far = event_df["events"][matching_json]["far"]
    if far > max_far:
        continue
    else:
        events_files.append(events_files_before_far_cut[i])

for i in range(len(events_files)):print(f"{i+1}. {events_files[i]} (i = {i})")

100%|██████████| 91/91 [00:00<00:00, 368.44it/s]

1. IGWN-GWTC2p1-v2-GW190803_022701_PEDataRelease_mixed_nocosmo.h5 (i = 0)
2. IGWN-GWTC3p0-v2-GW200208_130117_PEDataRelease_mixed_nocosmo.h5 (i = 1)
3. IGWN-GWTC3p0-v2-GW200112_155838_PEDataRelease_mixed_nocosmo.h5 (i = 2)
4. IGWN-GWTC3p0-v2-GW200316_215756_PEDataRelease_mixed_nocosmo.h5 (i = 3)
5. IGWN-GWTC2p1-v2-GW190925_232845_PEDataRelease_mixed_nocosmo.h5 (i = 4)
6. IGWN-GWTC3p0-v2-GW200115_042309_PEDataRelease_mixed_nocosmo.h5 (i = 5)
7. IGWN-GWTC2p1-v2-GW190708_232457_PEDataRelease_mixed_nocosmo.h5 (i = 6)
8. IGWN-GWTC2p1-v2-GW190521_030229_PEDataRelease_mixed_nocosmo.h5 (i = 7)
9. IGWN-GWTC2p1-v2-GW190701_203306_PEDataRelease_mixed_nocosmo.h5 (i = 8)
10. IGWN-GWTC2p1-v2-GW190720_000836_PEDataRelease_mixed_nocosmo.h5 (i = 9)
11. IGWN-GWTC2p1-v2-GW151226_033853_PEDataRelease_mixed_nocosmo.h5 (i = 10)
12. IGWN-GWTC2p1-v2-GW190828_065509_PEDataRelease_mixed_nocosmo.h5 (i = 11)
13. IGWN-GWTC2p1-v2-GW190929_012149_PEDataRelease_mixed_nocosmo.h5 (i = 12)
14. IGWN-GWTC2p1-v2-GW190425_08

In [6]:
def contains_pattern(s, options_list):
    combined_pattern = "|".join(options_list)
    return bool(re.search(combined_pattern, s))

if not os.path.exists(write_path + "event-list.txt"):
    with open(write_path + "event-list.txt", "w") as f:
        pass

In [7]:
re1 = r"GWTC-1"
re2= r"GWTC2p1"
re3 = r"GWTC3"

event_waveforms = []
compile_df = pd.DataFrame(columns=["event", "waveform", "cosmology", "distance_conversion", "prior_cosmology","prior_a_1 (or chi1)", "prior_a_2 (or chi2)", "prior_tilt_1", "prior_tilt_2"])

for i in tqdm(range(len(events_files))):
    file_path = events_files[i]
    event_file = folder_path+file_path
    with h5py.File(event_file,'r+') as file:
        wave_index = 0
        waveform = list(file.keys())[wave_index]
        if contains_pattern(event_file, ["GW200115", "GW190720", "GW190425", "GW190924", "GW190814", "GW190728", "GW190707", "GW170608"]):
            cosmology_used = file[waveform]["meta_data"]['meta_data']["cosmology"][:][0]
            cosmo_prior = "NOT INCLUDED (dl^2)"
            spin1_used = "NOT INCLUDED" ; spin2_used = "NOT INCLUDED"
            tilt1_used = "NOT INCLUDED" ; tilt2_used = "NOT INCLUDED"
        elif contains_pattern(event_file, ["GW170817"]):
            cosmology_used = "planck 2015"
            cosmo_prior = "NOT INCLUDED (dl^2)"
            spin1_used = "chi < 0.89 (Uniform)"; spin2_used = "chi < 0.89 (Uniform)"
            tilt1_used = "NOT INCLUDED (Isotropic)"; tilt2_used = "NOT INCLUDED (Isotropic)"
        else:
            cosmo_prior = file[waveform]["priors"]['analytic']["luminosity_distance"][:][0]
            cosmology_used = file[waveform]["meta_data"]['meta_data']["cosmology"][:][0]
            spin1_used = str(file[waveform]["priors"]['analytic']["a_1"][:][0]) ; spin2_used = str(file[waveform]["priors"]['analytic']["a_2"][:][0])
            tilt1_used = str(file[waveform]["priors"]['analytic']["tilt_1"][:][0]) ; tilt2_used = str(file[waveform]["priors"]['analytic']["tilt_2"][:][0])

    args = [f"{waveform}", f"{event_file}",
        "--Ho", f"{PLANCK_2018_Ho}", "--OmegaMatter", f"{PLANCK_2018_OmegaMatter}", "--OmegaRadiation",
        f"{PLANCK_2018_OmegaRadiation}", "--OmegaLambda", f"{PLANCK_2018_OmegaLambda}", "-o", f"{write_path}"]

    if re.search(re1, file_path):
        distance_conversion_used = "d_l^2"
        subprocess.run(["python", "h52csv_GWTC1.py", *args])
    elif re.search(re2, file_path):
        distance_conversion_used = "d_l^2"
        subprocess.run(["python", "h52csv_no_cosmo.py", *args])
    elif re.search(re3, file_path):
        distance_conversion_used = "d_l^2"
        subprocess.run(["python", "h52csv_no_cosmo.py", *args])
    else:
        raise ValueError("No match found")

    list_row = [file_path, waveform, cosmology_used, distance_conversion_used, cosmo_prior, spin1_used, spin2_used, tilt1_used, tilt2_used]
    compile_df.loc[i] = list_row

compile_df.to_csv("compile_df.csv", index=False)
compile_df

100%|██████████| 66/66 [01:12<00:00,  1.10s/it]


,event,waveform,cosmology,distance_conversion,prior_cosmology,prior_a_1 (or chi1),prior_a_2 (or chi2),prior_tilt_1,prior_tilt_2
0,IGWN-GWTC2p1-v2-GW190803_022701_PEDataRelease_...,C01:IMRPhenomXPHM,b'Planck15_lal',d_l^2,"b""PowerLaw(alpha=2, minimum=100, maximum=10000...","b""Uniform(minimum=0, maximum=0.99, name='a_1',...","b""Uniform(minimum=0, maximum=0.99, name='a_2',...","b""Sine(name='tilt_1', latex_label='$\\\\theta_...","b""Sine(name='tilt_2', latex_label='$\\\\theta_..."
1,IGWN-GWTC3p0-v2-GW200208_130117_PEDataRelease_...,C01:IMRPhenomXPHM,b'Planck15_lal',d_l^2,"b""PowerLaw(alpha=2, minimum=100, maximum=10000...","b""Uniform(minimum=0, maximum=0.99, name='a_1',...","b""Uniform(minimum=0, maximum=0.99, name='a_2',...","b""Sine(name='tilt_1', latex_label='$\\\\theta_...","b""Sine(name='tilt_2', latex_label='$\\\\theta_..."
2,IGWN-GWTC3p0-v2-GW200112_155838_PEDataRelease_...,C01:IMRPhenomXPHM,b'Planck15_lal',d_l^2,"b""PowerLaw(alpha=2, minimum=100, maximum=10000...","b""Uniform(minimum=0, maximum=0.99, name='a_1',...","b""Uniform(minimum=0, maximum=0.99, name='a_2',...","b""Sine(name='tilt_1', latex_label='$\\\\theta_...","b""Sine(name='tilt_2', latex_label='$\\\\theta_..."
3,IGWN-GWTC3p0-v2-GW200316_215756_PEDataRelease_...,C01:IMRPhenomXPHM,b'Planck15_lal',d_l^2,"b""PowerLaw(alpha=2, minimum=100, maximum=10000...","b""Uniform(minimum=0, maximum=0.99, name='a_1',...","b""Uniform(minimum=0, maximum=0.99, name='a_2',...","b""Sine(minimum=0, maximum=3.141592653589793, n...","b""Sine(minimum=0, maximum=3.141592653589793, n..."
4,IGWN-GWTC2p1-v2-GW190925_232845_PEDataRelease_...,C01:IMRPhenomXPHM,b'Planck15_lal',d_l^2,"b""PowerLaw(alpha=2, minimum=100, maximum=10000...","b""Uniform(minimum=0, maximum=0.99, name='a_1',...","b""Uniform(minimum=0, maximum=0.99, name='a_2',...","b""Sine(name='tilt_1', latex_label='$\\\\theta_...","b""Sine(name='tilt_2', latex_label='$\\\\theta_..."
...,...,...,...,...,...,...,...,...,...
61,IGWN-GWTC2p1-v2-GW190421_213856_PEDataRelease_...,C01:IMRPhenomXPHM,b'Planck15_lal',d_l^2,"b""PowerLaw(alpha=2, minimum=100, maximum=10000...","b""Uniform(minimum=0, maximum=0.99, name='a_1',...","b""Uniform(minimum=0, maximum=0.99, name='a_2',...","b""Sine(name='tilt_1', latex_label='$\\\\theta_...","b""Sine(name='tilt_2', latex_label='$\\\\theta_..."
62,IGWN-GWTC2p1-v2-GW190910_112807_PEDataRelease_...,C01:IMRPhenomXPHM,b'Planck15_lal',d_l^2,"b""PowerLaw(alpha=2, minimum=100, maximum=10000...","b""Uniform(minimum=0, maximum=0.99, name='a_1',...","b""Uniform(minimum=0, maximum=0.99, name='a_2',...","b""Sine(name='tilt_1', latex_label='$\\\\theta_...","b""Sine(name='tilt_2', latex_label='$\\\\theta_..."
63,IGWN-GWTC2p1-v2-GW190503_185404_PEDataRelease_...,C01:IMRPhenomXPHM,b'Planck15_lal',d_l^2,"b""PowerLaw(alpha=2, minimum=10, maximum=10000,...","b""Uniform(minimum=0, maximum=0.99, name='a_1',...","b""Uniform(minimum=0, maximum=0.99, name='a_2',...","b""Sine(name='tilt_1', latex_label='$\\\\theta_...","b""Sine(name='tilt_2', latex_label='$\\\\theta_..."
64,IGWN-GWTC3p0-v2-GW191129_134029_PEDataRelease_...,C01:IMRPhenomXPHM,b'Planck15_lal',d_l^2,"b""PowerLaw(alpha=2, minimum=100, maximum=10000...","b""Uniform(minimum=0, maximum=0.99, name='a_1',...","b""Uniform(minimum=0, maximum=0.99, name='a_2',...","b""Sine(minimum=0, maximum=3.141592653589793, n...","b""Sine(minimum=0, maximum=3.141592653589793, n..."


In [8]:
# read txt file
event_txt_files = []
with open(write_path + "event-list.txt", "r") as f:
    event_txt_files = f.readlines()

print("Number of events converted: ", len(event_txt_files))
print("Total number of events: ", len(events_files))

Number of events converted:  66
Total number of events:  66


In [9]:
# Read events and confirm the conversion.
import numpy as np
event_txt_files = []
with open(write_path + "event-list.txt", "r") as f:
    event_txt_files = f.read().splitlines()
event_txt_files.sort()
event_txt_files

['GW150914_095045.csv.gz',
 'GW151012_095443.csv.gz',
 'GW151226_033853.csv.gz',
 'GW170104_101158.csv.gz',
 'GW170608_020116.csv.gz',
 'GW170729_185629.csv.gz',
 'GW170809_082821.csv.gz',
 'GW170814_103043.csv.gz',
 'GW170817.csv.gz',
 'GW170818_022509.csv.gz',
 'GW170823_131358.csv.gz',
 'GW190408_181802.csv.gz',
 'GW190412_053044.csv.gz',
 'GW190413_134308.csv.gz',
 'GW190421_213856.csv.gz',
 'GW190425_081805.csv.gz',
 'GW190503_185404.csv.gz',
 'GW190512_180714.csv.gz',
 'GW190513_205428.csv.gz',
 'GW190517_055101.csv.gz',
 'GW190519_153544.csv.gz',
 'GW190521_030229.csv.gz',
 'GW190521_074359.csv.gz',
 'GW190527_092055.csv.gz',
 'GW190602_175927.csv.gz',
 'GW190620_030421.csv.gz',
 'GW190630_185205.csv.gz',
 'GW190701_203306.csv.gz',
 'GW190706_222641.csv.gz',
 'GW190707_093326.csv.gz',
 'GW190708_232457.csv.gz',
 'GW190720_000836.csv.gz',
 'GW190727_060333.csv.gz',
 'GW190728_064510.csv.gz',
 'GW190803_022701.csv.gz',
 'GW190814_211039.csv.gz',
 'GW190828_063405.csv.gz',
 'GW1908

In [10]:
import pandas as pd
import matplotlib.pyplot as plt

for i in tqdm(range(len(event_txt_files))):
    event = event_txt_files[i]
    df = pd.DataFrame(np.genfromtxt(write_path + event, delimiter=",", names=True))
    fig, ax = plt.subplots(nrows=2, ncols=4, figsize=(10, 4))
    ax[0, 0].hist(df["mass1_source"]); ax[0, 0].set_xlabel("mass1_source (Msun)")
    ax[0, 1].hist(df["mass2_source"]); ax[0, 1].set_xlabel("mass2_source (Msun)")
    ax[0, 2].hist(df["luminosity_distance"]); ax[0, 2].set_xlabel("luminosity_distance (MPc)")
    ax[0, 3].hist(df["redshift"]); ax[0, 3].set_xlabel("redshift (z)")
    ax[1, 0].hist(df["a_1"]); ax[1, 0].set_xlabel("a_1")
    ax[1, 1].hist(df["a_2"]); ax[1, 1].set_xlabel("a_2")
    ax[1, 2].hist(df["costilt1"]); ax[1, 2].set_xlabel("costilt1")
    ax[1, 3].hist(df["costilt2"]); ax[1, 3].set_xlabel("costilt1")
    plt.suptitle(event[:12] + f" - {df.shape[0]} samples")
    plt.tight_layout()
    plt.savefig("verify_PE/" + event[:12] + ".png")
    plt.close()


100%|██████████| 66/66 [00:23<00:00,  2.80it/s]


# Conversion of injections

In [11]:
vt_folder = "../../vt/"
vt_file = "o1+o2+o3_mixture_real+semianalytic-LIGO-T2100377-v2.hdf5"
vt_file_O3_full = "endo3_mixture-LIGO-T2100113-v12.hdf5"
vt_path = vt_folder + vt_file

In [12]:
with h5py.File(vt_folder + vt_file_O3_full, "r") as f:
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]
    print(type(f[a_group_key]))
    data = list(f[a_group_key])
    print(data)

with h5py.File(vt_path, "r") as f:
    print("Keys: %s" % f.keys())
    a_group_key = list(f.keys())[0]
    print(type(f[a_group_key]))
    data = list(f[a_group_key])
    print(data)

Keys: <KeysViewHDF5 ['injections']>
<class 'h5py._hl.group.Group'>
['declination', 'declination_sampling_pdf', 'detection_statistic_cwb', 'detection_statistic_mbta', 'detection_statistic_pycbc_bbh', 'detection_statistic_pycbc_hyperbank', 'distance', 'far_cwb', 'far_gstlal', 'far_mbta', 'far_pycbc_bbh', 'far_pycbc_hyperbank', 'gps_time', 'gps_time_int', 'ifar_cwb', 'ifar_gstlal', 'ifar_mbta', 'ifar_pycbc_bbh', 'ifar_pycbc_hyperbank', 'inclination', 'inclination_sampling_pdf', 'mass1', 'mass1_source', 'mass2', 'mass2_source', 'mixture_weight', 'optimal_snr_h', 'optimal_snr_l', 'optimal_snr_net', 'pastro_cwb', 'pastro_gstlal', 'pastro_mbta', 'pastro_pycbc_bbh', 'pastro_pycbc_hyperbank', 'polarization', 'redshift', 'right_ascension', 'right_ascension_sampling_pdf', 'sampling_pdf', 'spin1x', 'spin1y', 'spin1z', 'spin2x', 'spin2y', 'spin2z']
Keys: <KeysViewHDF5 ['injections']>
<class 'h5py._hl.group.Group'>
['far_cwb', 'far_gstlal', 'far_mbta', 'far_pycbc_bbh', 'far_pycbc_hyperbank', 'ifar_c

In [13]:
max_far = float(config["INJECTIONS"]["max_far"])

In [14]:
with h5py.File(vt_path,'r+') as file:
        waveform = list(file.keys())[0]
        event_waveforms.append(f"Converting {waveform} from {vt_path}")

args = ["--max-far", f"{max_far}",
        "--Ho", f"{PLANCK_2018_Ho}",
        "--OmegaMatter", f"{PLANCK_2018_OmegaMatter}",
        "--OmegaRadiation", f"{PLANCK_2018_OmegaRadiation}",
        "--OmegaLambda", f"{PLANCK_2018_OmegaLambda}",
        f"{vt_path}",
        # "-v",
        # f"{vt_folder}" + "sensitivity-estimate_O3.csv.gz"]
        f"{vt_folder}" + "sensitivity-estimate.csv.gz"]

subprocess.run(["python", "h52csv_injections.py", *args])
print("Done")

Done


In [15]:
# Read the data
f = h5py.File(vt_path, 'r')
cut = f['injections']['far_gstlal'][:] < 1
print(np.sum(f['injections']['mass2_source'][:] < 1.5))
np.sum(cut)

249624


165098

In [16]:
path = f"{vt_folder}" + "sensitivity-estimate.csv.gz"
df = pd.read_csv(path)
df # Checking inje# ctions
print(((df["mass2_source"] < 1.5) & (df["mass1_source"] < 1.5)).sum())
print(((df["mass2_source"] < 1.5)).sum())
df

2835
33924


,a_1,a_2,costilt1,costilt2,far,geocenter_time,lnprob_geocenter_time,lnprob_mass1_source_mass2_source_redshift_spin1spherical_spin2spherical,mass1_source,mass2_source,redshift,spin1x,spin1y,spin1z,spin2x,spin2y,spin2z
0,0.262990,0.353922,-0.443605,0.123831,2.461024e-05,0.0,1.0,-4.728567,2.149666,1.595966,0.075358,-0.233778,-0.030015,-0.116664,-0.319472,-0.145868,0.043827
1,0.192997,0.283679,-0.969152,-0.408359,1.772536e-05,0.0,1.0,-9.041965,7.257794,2.335832,0.137468,-0.007892,-0.046908,-0.187043,0.214222,-0.145476,-0.115843
2,0.226059,0.229092,0.637536,-0.725247,6.661743e-14,0.0,1.0,-5.480601,2.364014,1.740568,0.050528,-0.170921,-0.033432,0.144120,0.082180,-0.134627,-0.166148
3,0.902902,0.189674,0.265857,-0.723968,1.476506e-05,0.0,1.0,-9.121858,3.141577,1.072391,0.045338,-0.043686,0.869311,0.240043,0.013434,0.130152,-0.137318
4,0.711184,0.794992,0.815303,0.094001,1.526518e-09,0.0,1.0,-18.592321,71.112717,57.015984,0.719476,0.292093,0.290278,0.579830,-0.781097,0.127732,0.074730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202830,0.712716,0.315726,0.854941,0.213853,2.293210e-01,0.0,1.0,-9.912697,6.708300,2.146488,0.076519,-0.257178,0.265595,0.609330,0.103421,0.290565,0.067519
202831,0.203080,0.343764,-0.759649,-0.755698,3.966578e-09,0.0,1.0,-6.907011,2.082222,1.676427,0.024144,0.029651,0.128698,-0.154270,-0.011663,0.224836,-0.259782
202832,0.428977,0.396957,-0.199208,-0.569394,3.807962e-52,0.0,1.0,-12.395740,9.539573,2.426798,0.032087,-0.389226,0.158813,-0.085456,-0.249228,-0.210649,-0.226025
202833,0.631990,0.473451,-0.159549,0.681661,1.050514e-06,0.0,1.0,-12.923044,3.403745,2.128665,0.047793,0.316157,-0.537856,-0.100833,0.245378,0.244518,0.322733
